In [2]:
%load_ext autoreload


In [264]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
sys.path.append('../../')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
from ast import literal_eval
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/ddi"
outdir = 'pre-processed/original/'
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.converters.converter_ddi import get_dataset_dataframe, write_dataframe, \
read_dataframe, check_equality_of_written_and_read_df, write_into_txt, combine


In [231]:
df_train_drugbank = get_dataset_dataframe(res('Train/DrugBank/'), relation_extraction=True)

  0%|          | 0/569 [00:00<?, ?it/s]

total_files_to_read: 569  from dir:  /data/medg/misc/geeticka/relation_extraction/ddi/Train/DrugBank/


100%|██████████| 569/569 [06:33<00:00,  1.45it/s]


In [232]:
df_train_medline = get_dataset_dataframe(res('Train/MedLine/'), relation_extraction=True)

  3%|▎         | 4/142 [00:00<00:04, 32.56it/s]

total_files_to_read: 142  from dir:  /data/medg/misc/geeticka/relation_extraction/ddi/Train/MedLine/


100%|██████████| 142/142 [00:24<00:00,  5.77it/s]


In [233]:
df_test_drugbank = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'), relation_extraction=True)

  2%|▏         | 3/158 [00:00<00:05, 26.18it/s]

total_files_to_read: 158  from dir:  /data/medg/misc/geeticka/relation_extraction/ddi/Test/test_for_ddi_extraction_task/DrugBank/


100%|██████████| 158/158 [01:20<00:00,  1.97it/s]


In [234]:
df_test_medline = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'), relation_extraction=True)

  9%|▉         | 3/33 [00:00<00:01, 16.60it/s]

total_files_to_read: 33  from dir:  /data/medg/misc/geeticka/relation_extraction/ddi/Test/test_for_ddi_extraction_task/MedLine/


100%|██████████| 33/33 [00:05<00:00,  6.42it/s]


In [235]:
len(df_train_drugbank)

19687

In [236]:
len(df_train_medline)

1546

In [237]:
len(df_test_drugbank)

4308

In [238]:
len(df_test_medline)

388

## Testing for Empty Entities

In [239]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        metadata = row.metadata
        e1 = metadata['e1']['word_index']
        e2 = metadata['e2']['word_index']
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        original_sentence = df.iloc[index].original_sentence
        tokenized_sentence = df.iloc[index].tokenized_sentence
        metadata = df.iloc[index].metadata
        empty_rows_array.append([index, original_sentence, e1, e2, metadata, tokenized_sentence])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'original_sentence' , 'e1', 'e2', 'metadata', 'tokenized_sentence'])
    return empty_rows_array, new_df

In [240]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [241]:
get_empty_vals(df_train_drugbank)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [242]:
get_empty_vals(df_train_medline)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [243]:
get_empty_vals(df_test_drugbank)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [244]:
get_empty_vals(df_test_medline)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

## Write into csv format

In [245]:
if not os.path.exists(res(outdir)):
    os.makedirs(res(outdir))

In [246]:
write_dataframe(df_train_drugbank, res(outdir + 'train_drugbank_original.csv'))

In [247]:
df_train_drugbank_copy = read_dataframe(res(outdir + 'train_drugbank_original.csv'))

In [248]:
# The first checks with the pd.equals method, and the other does a manual checking per column
check_equality_of_written_and_read_df(df_train_drugbank, df_train_drugbank_copy)


(True, True)

In [249]:
write_dataframe(df_train_medline, res(outdir + 'train_medline_original.csv'))

In [250]:
df_train_medline_copy = read_dataframe(res(outdir + 'train_medline_original.csv'))

In [251]:
check_equality_of_written_and_read_dfuality_of_written_and_read_df_equality_of_written_and_read_dfk_equality_of_written_and_read_df(df_train_medline, df_train_medline_copy)

(True, True)

In [252]:
write_dataframe(df_test_drugbank, res(outdir + 'test_drugbank_original.csv'))

In [253]:
df_test_drugbank_copy = read_dataframe(res(outdir + 'test_drugbank_original.csv'))

In [254]:
check_equality_of_written_and_read_df(df_test_drugbank, df_test_drugbank_copy)

(True, True)

In [255]:
write_dataframe(df_test_medline, res(outdir + 'test_medline_original.csv'))

In [256]:
df_test_medline_copy = read_dataframe(res(outdir + 'test_medline_original.csv'))

In [257]:
check_equality_of_written_and_read_df(df_test_medline, df_test_medline_copy)

(True, True)

## Write into txt format

In [258]:
write_into_txt(df_train_drugbank, res(outdir + 'train_drugbank_original.txt'))

Unique relations: 	 ['effect' 'none' 'advise' 'mechanism' 'int']


In [259]:
write_into_txt(df_train_medline, res(outdir + 'train_medline_original.txt'))

Unique relations: 	 ['none' 'mechanism' 'effect' 'advise' 'int']


In [260]:
write_into_txt(df_test_drugbank, res(outdir + 'test_drugbank_original.txt'))

Unique relations: 	 ['int' 'effect' 'none' 'mechanism' 'advise']


In [261]:
write_into_txt(df_test_medline, res(outdir + 'test_medline_original.txt'))

Unique relations: 	 ['none' 'effect' 'mechanism' 'advise' 'int']


## Combine the train/test data of medline and drugbank

In [266]:
combine(res, outdir, 'train_medline_original', 'train_drugbank_original', 'train_original.txt')

In [267]:
combine(res, outdir, 'test_medline_original', 'test_drugbank_original', 'test_original.txt')